In [4]:
! pip install pywikibot
! pip install mwparserfromhell
! pip install watchdog

In [5]:
import os

os.environ['PYWIKIBOT_DIR'] = './wiki_reader/'

In [6]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, StructField, StructType, ArrayType, LongType, DoubleType

from spark_app.scorers import score_text
from spark_app.spark_tools import SparkSentimentStreamer
from pathlib import Path

## Input

In [7]:
request = 'peano'

batch_size = 100
limit = None
preload_content = True
is_category = False

wiki_page_dir = 'requests/'
spark_results_dir = 'responses/'

## Processing

In [6]:
# from ml.LogisticRegressionCached import readFromCache

# (lrModel, pipelineModel) = readFromCache('./ml/train/')

# def score_text_ml(text):
#     df = spark.createDataFrame([(text, 2)], ['text', 'target'])
#     df_transformed = pipelineModel.transform(df) # To fix
#     predictions = lrModel.transform(df_transformed)
#     predictions = predictions.select(['text', 'probability', 'prediction'])
#     pd_predictions = predictions.toPandas()
#     positive_probability = pd_predictions.iloc[0]['probability'][1]
#     overall_probability = 2 * positive_probability - 1
    
#     return overall_probability

In [13]:
from ml.OutputProcessing import process_output_files

# process_output_files(spark_results_dir, 1000)

IntProgress(value=0, description='Iter 0/1000', max=1000)

'Mean: 0.256'

,title,url,sentiment
0,Page #1,url,0.352
1,Page #0,url,0.160


In [9]:
def spark_process(request, score_func):
    spark = SparkSession.builder\
        .master("local[*]")\
        .appName("NetworkWordCount")\
        .getOrCreate()
    
    sc = spark.sparkContext
    ssc = StreamingContext(sc, 1)
    
    dataInp = wiki_page_dir + request
    dataOut = spark_results_dir + request
    #TODO: check if output module can track yet uncreacted directory or if spark could output into existing dir
#     Path(dataOut).mkdir(parents=True, exist_ok=True)
    
    streamer = SparkSentimentStreamer(sc, ssc, spark, score_func, dataInp, dataOut)
    streamer.run()
    streamer.stop()

In [10]:
from concurrent.futures import ThreadPoolExecutor
import wiki_reader.reader as reader

wiki_wrapper = lambda r,b,l,cat,cont: reader.query(r,out_dir=wiki_page_dir,batch_size=b,
                                              limit=l,is_category=cat,preload_content=cont)

query_size = reader.query_size(request)

In [14]:
with ThreadPoolExecutor(max_workers=2) as e:
    e.submit(wiki_wrapper, request, batch_size, limit, is_category, preload_content)
    e.submit(spark_process, request, score_text)
    # e.submit(process_output_files, spark_results_dir, query_size)

Cleaning old data
Downloading...
Dumped 100 pages | peano
Dumped 200 pages | peano
+--------------------+--------------------+--------------------+
|                text|               title|                 url|
+--------------------+--------------------+--------------------+
|0 (zero) is both ...|                   0|https://en.wikipe...|
|1 (one, also call...|                   1|https://en.wikipe...|
|thumb|right| Flow...|           Algorithm|https://en.wikipe...|
|thumb|right|Giuse...|Arithmetices prin...|https://en.wikipe...|
|In mathematics, a...|    Axiomatic system|https://en.wikipe...|
+--------------------+--------------------+--------------------+
only showing top 5 rows

+--------------------+--------------------+---------+
|               title|                 url|sentiment|
+--------------------+--------------------+---------+
|                   0|https://en.wikipe...|      1.0|
|                   1|https://en.wikipe...|      1.0|
|           Algorithm|https://en.wiki

KeyboardInterrupt: 